In [100]:
class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, do_ifactor=False, anti_windup_backcalc=True):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self.itime = itime
        if self.itime:
            self.oneoveritime = 1 / self.itime
        else:
            self.oneoveritime = 0
        self.dtime = dtime
        self.dif = do_ifactor
        self.awb = anti_windup_backcalc

        self.accumulated_error = 0
        self.bump = 0
        self.last_output = 0
        self.last_error = 0
        self.last_pv = 0

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        err = sp - pv
        uk0 = self.pgain * err
        self.bump = op - uk0
        
    set_bump = man_to_auto

    def reset(self):
        self.accumulated_error = 0
        self.last_pv = 0
        self.last_error = 0        
    
    def step(self, pv, sp, dt=1):
        err = sp - pv
        dpv = pv - self.last_pv
        ierr = (err + self.last_error) / 2 * dt
        
        if self.dif:
            ierr *= 1 / (1 + err * err / 250)  # Labview's ifactor thingy
        
        # bump (aka controller bias) isn't normally
        # included in Up, but no one else reads my 
        # code :)
        Up = self.bump + self.pgain * err
        Ui = self.oneoveritime * (ierr + self.accumulated_error) * self.pgain
        Ud = self.dtime * dpv * self.pgain
        Uk = Up + Ui + Ud
        
        # XXX debugging
        self.Ui = Ui * self.pgain
        self.Uk = Uk
        self.Ud = Ud * self.pgain
        self.Up = Up
        
        # Coercion & back calculation      
        back_calc = False
        if Uk > self.auto_max:
            Uk = self.auto_max
            back_calc = True
            am = self.auto_max
        elif Uk < self.auto_min:
            Uk = self.auto_min
            back_calc = True
            am = self.auto_min
            
        if back_calc and self.awb:
            ierr = self.itime / self.pgain * (am - Up - Ud) - self.accumulated_error
            
        self.accumulated_error += ierr
        self.last_output = Uk
        self.last_pv = pv
        self.last_error = err
        self.last_ierr = ierr
        
        return Uk

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f AccumError: %.4f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime,
                                                                          self.accumulated_error)
    __str__ = __repr__
    

In [2]:
def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.off_to_auto(pv, sp)
        res_op = p.step(pv, sp)
        assert_equal(p.last_error, sp)
        assert_equal(p.accumulated_error, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [3]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

In [4]:
from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

In [5]:
from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]

In [6]:
from officelib.xllib import *
xl = Excel()

wb = xl.ActiveWorkbook
wb = wb or xl.Workbooks.Add()

ws = xl.ActiveSheet
ws = ws or wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [7]:

class TempProcess():
    def __init__(self, delay, initial=20, env=18.5, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial).cycle
        self.g = g / 60
        self.k = k / 60
        self.env = env
        
    def step(self, pv, op):
        op = self.tdelay(op)
        dT = pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        return pv + dpv

In [54]:
def temp_sim(p, i, d, delay, amax, amin, end, op=0, pv=20, sp=37, 
             g=0.0019254, k=-0.001579031, mode='m2a'):
    pid = PIDController(p,m2s(i),d,amax,amin, do_ifactor=True)
    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = TempProcess(delay, op, g=g, k=k)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t, pv, op))
        if t >= end:
            break
    return data
    

In [77]:
def paste(cells, data, offset=0):
    cells.Range(cells(1,1+offset), cells(1, 3+offset)).Value = [("T", "PV", "OP")]
    topleft = cells(2,1+offset)
    bottomright = topleft.Offset(len(data), len(data[0]))
    cells.Range(topleft, bottomright).Clear()
    cells.Range(topleft, bottomright).Value = data
    
def clear(cells, data, offset=0):
    topleft = cells(2,1+offset)
    bottomright = topleft.Offset(len(data), len(data[0]))
    cells.Range(topleft, bottomright).Clear()

In [176]:
offset = 0
p = 60
i = 35

In [168]:
clear(cells, data, 0)
data = temp_sim(p=p, i=i, d=0, delay=m2s(4.7), 
                amax=100, amin=0, end=m2s(600), op=0, 
                pv=23, sp=37, g=0.0019254*50/80, k=-0.001579031*50/80, mode='a2a')
paste(cells, data, 0)

In [169]:
clear(cells, data, 0)
data = temp_sim(p=p, i=i, d=0, delay=m2s(4.7), 
                amax=100, amin=0, end=m2s(600), op=0, 
                pv=23, sp=37)
paste(cells, data)

In [177]:
clear(cells, data, 0)
data = temp_sim(p=p, i=i, d=0, delay=m2s(4.7), 
                amax=100, amin=0, end=m2s(300), op=15, 
                pv=38, sp=39, mode='a2a')
paste(cells, data)

In [124]:
print("10/12/17 3:16:45")
print("1484254382.720792")

10/12/17 3:16:45
1484254382.720792


In [126]:
import datetime
d = datetime.datetime.now()
d.strftime("%m/%d/%y %H:%M:%S")

'01/12/17 13:52:09'